# 4 start FIWARE pods on Azure AKS

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/core"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/core
```

## load environment variables

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

## start fiware orion on AKS
[fiware orion](https://catalogue-server.fiware.org/enablers/publishsubscribe-context-broker-orion-context-broker)

In [ ]:
kubectl apply -f orion/orion-service.yaml

In [ ]:
kubectl apply -f orion/orion-deployment.yaml

In [ ]:
kubectl get pods -l app=orion

example)
```
NAME                     READY     STATUS    RESTARTS   AGE
orion-54f5cdcb5d-d2pt5   1/1       Running   0          56s
orion-54f5cdcb5d-hv274   1/1       Running   0          56s
orion-54f5cdcb5d-xbnx2   1/1       Running   0          56s
```

In [ ]:
kubectl get services -l app=orion

example)
```
NAME      TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
orion     ClusterIP   10.0.44.126   <none>        1026/TCP   1m
```

In [ ]:
curl -i https://api.${DOMAIN}/orion/v2/entities/

example)
```
HTTP/1.1 401 Unauthorized
content-length: 60
content-type: text/plain
www-authenticate: Bearer realm="token_required"
date: Thu, 14 Feb 2019 06:32:37 GMT
server: envoy

{"authorized":false,"error":"missing Header: authorization"}
```

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" https://api.${DOMAIN}/orion/v2/entities/

example)
```
HTTP/1.1 200 OK
content-length: 2
content-type: application/json
fiware-correlator: 4731eb48-4dc1-11e8-b1a2-0a580af4010a
date: Wed, 02 May 2018 04:28:35 GMT
x-envoy-upstream-service-time: 5
server: envoy

[]
```

**If you can't get 200 OK, please restart all ambassador's pods like "ambassador-cf99ff6bb-45jpl", "ambassador-cf99ff6bb-8kht7" and "ambassador-cf99ff6bb-lv2tk"**

## start fiware idas(iotagent-ul) on AKS
[fiware IDAS(iotagent-ul)](https://catalogue-server.fiware.org/enablers/backend-device-management-idas)

### start idas

In [ ]:
env IOTA_PASSWORD=${MQTT__iotagent} envsubst < idas/rb-config.js > /tmp/rb-config.js
kubectl create secret generic iotagent-config --from-file /tmp/rb-config.js
rm /tmp/rb-config.js

In [ ]:
kubectl apply -f idas/iotagent-ul-service.yaml

In [ ]:
kubectl apply -f idas/iotagent-ul-deployment.yaml

In [ ]:
kubectl get pods -l app=iotagent-ul

example)
```
NAME                           READY     STATUS    RESTARTS   AGE
iotagent-ul-79685b64bf-8krps   1/1       Running   0          3m
iotagent-ul-79685b64bf-m6nlg   1/1       Running   0          3m
iotagent-ul-79685b64bf-mjpbl   1/1       Running   0          3m
```

In [ ]:
kubectl get services -l app=iotagent-ul

example)
```
NAME          TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)             AGE
iotagent-ul   ClusterIP   10.0.180.155   <none>        4041/TCP,7896/TCP   43s
```

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: test" -H "Fiware-Servicepath: /*" https://api.${DOMAIN}/idas/ul20/manage/iot/services/

example)
```
HTTP/1.1 200 OK
x-powered-by: Express
fiware-correlator: c114fc5e-b4a2-40f6-b7fe-1d68369784e5
content-type: application/json; charset=utf-8
content-length: 25
etag: W/"19-WMYe0U6ocKhQjp+oaVnMHLdbylc"
date: Wed, 02 May 2018 06:16:18 GMT
x-envoy-upstream-service-time: 9
server: envoy

{"count":0,"services":[]}
```

**If you can't get 200 OK, please restart all ambassador's pods like "ambassador-cf99ff6bb-45jpl", "ambassador-cf99ff6bb-8kht7" and "ambassador-cf99ff6bb-lv2tk"**

## start fiware cygnus (mongodb sink) on AKS
[fiware cygnus](https://catalogue-server.fiware.org/enablers/cygnus)

### start cygnus-mongo

In [ ]:
kubectl apply -f cygnus/cygnus-mongo-service.yaml

In [ ]:
kubectl apply -f cygnus/cygnus-mongo-deployment.yaml

In [ ]:
kubectl get pods -l app=cygnus-mongo

example)
```
NAME                            READY     STATUS    RESTARTS   AGE
cygnus-mongo-588cb8cb46-42bcn   1/1       Running   0          1m
cygnus-mongo-588cb8cb46-k9cr2   1/1       Running   0          1m
cygnus-mongo-588cb8cb46-phvst   1/1       Running   0          1m
```

In [ ]:
kubectl get services -l app=cygnus-mongo

example)
```
NAME           TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)             AGE
cygnus-mongo   ClusterIP   10.0.10.63   <none>        5050/TCP,8081/TCP   1m
```